In [118]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [119]:
spark=SparkSession.builder.appName('Demo for better understanding').getOrCreate()

In [120]:
df=spark.read.csv('titanic_test.csv',header=True)

In [121]:
type(df)

pyspark.sql.dataframe.DataFrame

In [122]:
df.show()

+------------+------+--------------------+------+------+-----+-----+----------------+--------+-----+--------+----+-----+--------------------+
|passenger_id|pclass|                name|   sex|   age|sibsp|parch|          ticket|    fare|cabin|embarked|boat| body|           home.dest|
+------------+------+--------------------+------+------+-----+-----+----------------+--------+-----+--------+----+-----+--------------------+
|         295|     1|Thayer, Mr. John ...|  male|  17.0|    0|    2|           17421|110.8833|  C70|       C|   B| null|       Haverford, PA|
|        1150|     3|Risien, Mr. Samue...|  male|  null|    0|    0|          364498|    14.5| null|       S|null| null|                null|
|          89|     1|Davidson, Mr. Tho...|  male|  31.0|    1|    0|      F.C. 12750|    52.0|  B71|       S|null| null|        Montreal, PQ|
|        1063|     3|Nirva, Mr. Iisakk...|  male|  41.0|    0|    0|SOTON/O2 3101272|   7.125| null|       S|null| null| Finland Sudbury, ON|
|     

In [123]:
filteredDF=df.drop('home.dest','ticket','body')

In [124]:
for c in filteredDF.columns:
    print(col(c))

Column<'passenger_id'>
Column<'pclass'>
Column<'name'>
Column<'sex'>
Column<'age'>
Column<'sibsp'>
Column<'parch'>
Column<'fare'>
Column<'cabin'>
Column<'embarked'>
Column<'boat'>


In [125]:
filteredDF.printSchema()

root
 |-- passenger_id: string (nullable = true)
 |-- pclass: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- sibsp: string (nullable = true)
 |-- parch: string (nullable = true)
 |-- fare: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- boat: string (nullable = true)



In [126]:
typecastedDF=filteredDF.select(
    *(
        col(c).cast('float') if ('p' in c) else col(c) for c in filteredDF.columns 
    )
)

In [127]:
from pyspark.sql.functions import count,when,isnan

In [128]:
typecastedDF.select(
    [
        count(when(col(c).isNull(),c)) for c in typecastedDF.columns
    ]
).show()

+-------------------------------------------------------------+-------------------------------------------------+---------------------------------------------+-------------------------------------------+-------------------------------------------+-----------------------------------------------+-----------------------------------------------+---------------------------------------------+-----------------------------------------------+-----------------------------------------------------+---------------------------------------------+
|count(CASE WHEN (passenger_id IS NULL) THEN passenger_id END)|count(CASE WHEN (pclass IS NULL) THEN pclass END)|count(CASE WHEN (name IS NULL) THEN name END)|count(CASE WHEN (sex IS NULL) THEN sex END)|count(CASE WHEN (age IS NULL) THEN age END)|count(CASE WHEN (sibsp IS NULL) THEN sibsp END)|count(CASE WHEN (parch IS NULL) THEN parch END)|count(CASE WHEN (fare IS NULL) THEN fare END)|count(CASE WHEN (cabin IS NULL) THEN cabin END)|count(CASE WHEN (embarked

In [129]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age'],
    outputCols=['age']
)
model=imputer.fit(typecastedDF)

imputed_data=model.transform(typecastedDF)

imputed_data.fillna('')

imputed_data.select(
    [
        count(when(col(c).isNull(),c)) for c in imputed_data.columns
    ]
).show()


IllegalArgumentException: requirement failed: Column age must be of type numeric but was actually of type string.

In [130]:
filteredDF=imputed_data.drop('passenger_id','Cabin','Embarked','Name','sex')
filteredDF.show()

+------+---------+-----+-----+--------+----+
|pclass|      age|sibsp|parch|    fare|boat|
+------+---------+-----+-----+--------+----+
|   1.0|     17.0|  0.0|  2.0|110.8833|   B|
|   3.0|30.541216|  0.0|  0.0|    14.5|null|
|   1.0|     31.0|  1.0|  0.0|    52.0|null|
|   3.0|     41.0|  0.0|  0.0|   7.125|null|
|   3.0|     21.0|  0.0|  0.0|  7.8958|null|
|   3.0|   0.3333|  0.0|  2.0|    14.4|null|
|   2.0|     37.0|  1.0|  0.0|    26.0|null|
|   3.0|     15.0|  0.0|  0.0|   7.225|   C|
|   2.0|     14.0|  0.0|  0.0|    65.0|null|
|   1.0|     47.0|  0.0|  0.0|    52.0|null|
|   2.0|     44.0|  1.0|  0.0|    26.0|null|
|   2.0|     60.0|  1.0|  1.0|    39.0|null|
|   2.0|     43.0|  0.0|  1.0|    21.0|null|
|   1.0|     61.0|  0.0|  0.0| 32.3208|null|
|   3.0|30.541216|  0.0|  0.0|  7.8958|null|
|   2.0|30.541216|  0.0|  0.0|     0.0|null|
|   2.0|      6.0|  0.0|  1.0|    33.0|  11|
|   1.0|     36.0|  1.0|  0.0|   78.85|null|
|   3.0|30.541216|  0.0|  0.0|  7.8958|null|
|   2.0|  

ModuleNotFoundError: No module named 'splearn.preprocessing'